Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox,ttk
from tqdm import tqdm
import joblib  # For loading saved models (assuming your model is saved as a .pkl or similar file)
from sklearn.preprocessing import StandardScaler
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import tkinter as tk
from tkinter import simpledialog, messagebox
import os

In [ ]:
import time

start_time = None

Save Prediction Results into The Downloads of a Local System

In [ ]:
from tkinter import messagebox
from pathlib import Path

def save_predictions_directly(predictions_df):
    
    # Replace 1 with 'Respondent' and 0 with 'Non-Respondent'
    predictions_df['Prediction'] = predictions_df['Prediction'].replace({1: 'Respondent', 0: 'Non-Respondent'})
    
    # Get the default Downloads folder
    downloads_path = str(Path.home() / "Downloads")
    output_file = os.path.join(downloads_path, "predictions.csv")
    
    # Save to CSV
    predictions_df.to_csv(output_file, index=False)
    messagebox.showinfo("Success", f"Prediction completed! File saved to: {output_file}")

In [ ]:
# Global variable to store the prediction data
predict_data = None
normalization_confirmed = False  # Global flag to track whether the data is confirmed as normalized

# Initialize the scaler globally
scl = StandardScaler()

# List of specified gene IDs
specified_gene_ids = ['701', '890', '1869', '2187', '2491', '2556', '3619', '4085', '4283', '5111', '5426', '5557', '5984', '6502', 
                      '6529', '7298', '9133', '9700', '10036', '10090', '10403', '10615', '10959', '10992', '11339', '22983', 
                      '23397', '23649', '24137', '27101', '51514', '55143', '55355', '55388', '55723', '57149', '78995', '79733', 
                      '83540', '83903', '84318', '91687', '93323', '113130', '195828', '284367', '285643', '541468', '643677']
# Reference mean and standard deviation for feature '701' in the training set
reference_mean = 6.16178
reference_std = 0.9687

Log2(TPM+1) transformation function for Normalization

In [ ]:
# Log2(TPM+1) transformation function
def log2p1(x):
    return np.log2(x + 1)

In [ ]:
# Function to normalize RNA-seq data using gene lengths
def normalize_rna_seq_data(data, Org_data):
    global normalize
    try:      
        
        # Step 3: Transpose the RNA-seq feature selected data
        gene_list_2021_train = np.transpose(data)
        gene_list_2021_train.index.name = 'Entrez'
        gene_list = gene_list_2021_train.index.tolist()

        # Step 3: Transpose the RNA-seq full data
        gene_list_2021_train1 = np.transpose(Org_data)
        gene_list_2021_train1.index.name = 'Entrez'
        gene_list1 = gene_list_2021_train1.index.tolist()
      
        gene_length_file_path = r'C:\GUI\Python\fData.csv' # Change the path of this file
        
        # Load the gene length data
        data_gene_length = pd.read_csv(gene_length_file_path)
        print(f"Gene length data loaded successfully, shape: {data_gene_length.shape}")

        # Step 3: Standardize Entrez IDs in gene length data
        data_cleaned = data_gene_length.dropna(subset=['Entrez'])
        data_cleaned['Entrez'] = data_cleaned['Entrez'].astype(int)
     
        # Convert gene_list to int64 for comparison
        gene_list = [np.int32(gene) for gene in gene_list]
        data_gene_length = data_cleaned[data_cleaned['Entrez'].isin(gene_list)]

        gene_list1 = [np.int32(gene) for gene in gene_list1]
        data_gene_length1 = data_cleaned[data_cleaned['Entrez'].isin(gene_list1)]

        duplicated_rows = data_gene_length[data_gene_length.duplicated(subset='Entrez')]
        data_gene_length = data_gene_length.drop_duplicates(subset='Entrez', keep='first')

        duplicated_rows1 = data_gene_length1[data_gene_length1.duplicated(subset='Entrez')]
        data_gene_length1 = data_gene_length1.drop_duplicates(subset='Entrez', keep='first')
        
        
        # Convert 'Entrez' to a categorical type with a specific ordering
        data_gene_length['Entrez'] = pd.Categorical(data_gene_length['Entrez'], categories = gene_list, ordered=True)
        # Sort data_gene_length by the 'Entrez' column based on the specified order
        data_gene_length = data_gene_length.sort_values('Entrez')
        #print("data_gene_length is: ", data_gene_length.shape)


        # Convert 'Entrez' to a categorical type with a specific ordering
        data_gene_length1['Entrez'] = pd.Categorical(data_gene_length1['Entrez'], categories = gene_list1, ordered=True)
        # Sort data_gene_length by the 'Entrez' column based on the specified order
        data_gene_length1 = data_gene_length1.sort_values('Entrez')
        #print("data_gene_length1 is: ", data_gene_length1.shape)

        ################################################################################################################

        # Step 5: Create a progress bar
        progress_window = tk.Toplevel(root)
        progress_window.title("Normalization Progress")
        progress_label = tk.Label(progress_window, text="Normalization in Progress...", font=("Times New Roman", 14))
        progress_label.grid(pady=10)
        progress_bar = ttk.Progressbar(progress_window, length=400, mode='determinate')
        progress_bar.grid(pady=20)
        
        total_steps = len(data_gene_length) + len(data_gene_length1)
        progress_bar['maximum'] = total_steps

        #*************************************************************************************************************
        
        data_gene_TPM1 = gene_list_2021_train1.copy()
        data_gene_TPM = gene_list_2021_train.copy()
                        
        # Step 4: Normalize the subset using the calculated denominator
        for i in tqdm(range(len(data_gene_TPM1))):
            data_gene_TPM1.iloc[i] = data_gene_TPM1.iloc[i] / data_gene_length1.iloc[i]['length']
            progress_bar['value'] += 1
            progress_window.update()
            
        for i in tqdm(range(len(data_gene_TPM))):
            data_gene_TPM.iloc[i] = data_gene_TPM.iloc[i] / data_gene_length.iloc[i]['length']
            progress_bar['value'] += 1
            progress_window.update()
    
        # Step 5: Apply normalization to subset      
        data_gene_TPM = data_gene_TPM / data_gene_TPM1.sum() * 1e6
            
        # Optional: Apply log2(x+1) transformation if needed
        log2TPMp1 = np.vectorize(log2p1)(data_gene_TPM)
        GUI_data_gene_log2TPMp1 = data_gene_TPM.copy()
        GUI_data_gene_log2TPMp1.iloc[:, :] = log2TPMp1
    
        print("Normalization on subset completed successfully.")
        output_file = r"C:\GUI\Python\Normalized.csv"  # Change the the path to save into your local system

        #**************************************************************************************************************

        # Make sure the Entrez column is included in the data
        if 'Entrez' not in GUI_data_gene_log2TPMp1.columns:
            if len(gene_list) == len(GUI_data_gene_log2TPMp1):
                GUI_data_gene_log2TPMp1 = np.transpose(GUI_data_gene_log2TPMp1)
                
        else:
            raise ValueError("Length of Entrez IDs does not match the data length.")       
            
        # Save data including the Entrez column to the CSV file
        if output_file:
            GUI_data_gene_log2TPMp1.to_csv(output_file, index=False)  # Ensure index is not saved if you don't need it
            print(f"Data with Entrez saved successfully to {output_file}")
            messagebox.showinfo("Success", "Log(TPM+1) completed and saved successfully!")
        progress_window.destroy()

        GUI_data_gene_log2TPMp2 = np.transpose(GUI_data_gene_log2TPMp1)
        normalize = GUI_data_gene_log2TPMp1
        
    except Exception as e:
        messagebox.showerror("Error", str(e))


In [ ]:
def preprocess_data():
    start_time = time.time()
    global predict_data, specified_gene_ids, normalize
    try:
        
        # Step 1: Load the RNA-seq data file
        rna_seq_file_path = filedialog.askopenfilename(title="Select RNA-Seq Data File", 
                                                       filetypes=[("CSV Files", "*.csv")])
        if not rna_seq_file_path:
            messagebox.showerror("Error", "No RNA-seq file selected.")
            return  # Exit if no file is selected

        # Step 2: Load the RNA-seq data
        gene_data = pd.read_csv(rna_seq_file_path, index_col=0)
    
    
        cleaned_column_names = [str(col).strip() for col in gene_data.columns]

        # Update the DataFrame with cleaned column names
        gene_data.columns = cleaned_column_names
       #print("gene_data is: ", gene_data.head(5))

        # Check for matching gene IDs
        selected_columns = gene_data.columns.intersection(specified_gene_ids)
        

        # Debugging output to check if intersection worked correctly
        print("Matched Columns:", selected_columns.tolist())  # Debugging line

        if selected_columns.empty:
            messagebox.showwarning("Warning", "No matching gene IDs found in the data.")
            return  # Exit the function if no matching genes found

        # Select the data based on the matching gene IDs
        selected_data = gene_data[selected_columns]
        
        # Prompt the user to save the selected data
        output_file = r"C:\GUI\Python\Feature_Selection.csv"    # Change the the path to save into your local system
        selected_data.to_csv(output_file, index=False)
        messagebox.showinfo("Success", "Feature selection completed and saved successfully!")
        
        if '701' in selected_data.columns:
            feature_mean = selected_data['701'].mean()
            
                      
            # Check if mean is within range of reference mean ± reference std
            if abs(feature_mean - reference_mean) > reference_std:
                
                # If not normalized, apply StandardScaler normalization
                normalize_rna_seq_data(selected_data, gene_data)
                #messagebox.showinfo("Info", "Data was not normalized. Normalization has been applied.")
            else:
                messagebox.showinfo("Info", "Data is already normalized.")
                normalize = selected_data
        
        else:
            messagebox.showerror("Error", "Feature '701' not found in the data.")

        # Store selected and normalized data for later prediction
        predict_data = normalize
        load_weights_and_predict()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during feature selection: {e}")
        

In [ ]:
# Function to apply the saved scaling parameters to new prediction data
def scale_to_existing_scale(x, scale_info):
    # Apply scaling using precomputed scale info
    return (x - scale_info['center']) / scale_info['scale']

def load_weights_and_predict():
    global predict_data, specified_gene_ids
    start_time = time.time()
    # Check if predict_data is None or empty
    if predict_data is None or predict_data.empty:
        messagebox.showerror("Error", "No prediction data available. Please load the data first.")
        return

    try:
        # Load the pre-trained model
        model_weights_file = r'C:\GUI\Python\best_logit_model.pkl'     # Change the path where the model weights are saved
        model = joblib.load(model_weights_file)

        # Load scaling parameters from the saved file
        scale_info = joblib.load(r'C:\GUI\Python\mUC_scaling_info.pkl')     # Change the path where the scaling file is saved
        

        # Check if the loaded object is a valid model by verifying if it has the 'predict' method
        if not hasattr(model, 'predict'):
            messagebox.showerror("Error", "Loaded model is not valid. It doesn't have a 'predict' method.")
            return

        # Extract only the numeric columns for scaling
        predict_data_numeric = predict_data.select_dtypes(include=[np.number])
            
        # Standardize the prediction data using the loaded scale_info
        X_Pred_scaled = scale_to_existing_scale(predict_data_numeric.values, scale_info)
        X_Pred_scaled_df = pd.DataFrame(X_Pred_scaled, columns=predict_data.columns)
        X_Pred_scaled_df.to_csv(r'C:\GUI\Python\GUIScaledtestset.csv', index=False)
        
        # Make predictions using the trained model
        predictions = model.predict(X_Pred_scaled)
       
        # data_train = pd.read_csv(r'C:\Spandana\GUI\Python_Train\Final_File\TPM\LR_Model_Training\Training_set\log2TPM_training_FS_beforeStanberdization.csv')
        df = pd.read_csv(r'C:\GUI\Python\log2TPMp1_test.csv')    #Change the path of this file
        df.columns.values[0] = 'SampleID'
        sample_ids = df['SampleID'].tolist()
       
        # Ensure both arrays are of the same length
        if len(sample_ids) != len(predictions):
            raise ValueError("Mismatch between the number of Sample IDs and Predictions")

        # Print the lengths of the arrays to debug
        print(f"Number of Sample IDs: {len(sample_ids)}")
        print(f"Number of Predictions: {len(predictions)}")

        
        predictions_df = pd.DataFrame({
            "SampleID": sample_ids,
            "Prediction": predictions
        })
       
        # Replace 1 with 'Respondent' and 0 with 'Non-Respondent'
        predictions_df['Prediction'] = predictions_df['Prediction'].replace({1: 'Respondent', 0: 'Non-Respondent'})


        messagebox.showinfo("Success", "Prediction completed and saved successfully!")
       
        save_predictions_directly(predictions_df)

        end_time = time.time()
        total_time = end_time - start_time
        messagebox.showinfo("Time Taken", f"Total processing time: {total_time:.2f} seconds")
        print(f"Total time from start of processing to prediction completion: {total_time:.2f} seconds")
       
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")


In [26]:
root = tk.Tk()
root.title("Drug Response Prediction")
root.geometry("750x500")
root.configure(bg="light blue")

 # Configure a grid layout with a single centered column
root.grid_columnconfigure(1, weight=1)  # This centers elements in column 0


# Main Label
title_label = tk.Label(root, text="Drug Response Prediction Tool", font=("Times New Roman", 18, 'bold'))
title_label.grid(row=0, column=1, columnspan=5, pady=30)  # Center title


# Test Data button with color
test_data_button = tk.Button(
    root, 
    text="Upload File", 
    font=("Times New Roman", 14, 'bold'), 
    command=preprocess_data,
    bg="#a084ca",  # Background color
    fg="black"       # Text color
)
test_data_button.grid(row=55, column=1, pady=50, padx=50, ipadx=50) 

root.mainloop()

time.sleep(1)